
# Advanced Modeling and Simulation Course

### Instructor: Dr. M. Nedim ALPDEMİR

*This material is prepared as part of the EE592-Advanced Modeling And Simulation course given at Yıldırım Beyazıd University (YBÜ).*

# Discrete Event Simulation Example - 2

## Bank  example

### Covers:

- Resources: Resource
- Condition events

### Scenario:

  * The bank branch has a counter with a random service time
  * customers arrive in random intervals. The distribution used for modeling the cutomer arrivals should be  exponential
  * customers give up after waiting some time in the queue. 
  * The branch manager want to know how many customers quit from the branch in a typical day
  


In [29]:
import random

import simpy


RANDOM_SEED = 42
NEW_CUSTOMERS = 15  # Total number of customers
INTERVAL_CUSTOMERS = 2.0  # Generate new customers roughly every x seconds
MIN_PATIENCE = 1  # Min. customer patience
MAX_PATIENCE = 2  # Max. customer patience


def CustomerGenerator(env, number, interval, serv_desk):
    """This generates customers randomly"""
    for i in range(number):
        c = customer(env, 'Customer%02d' % i, serv_desk, time_in_bank=12.0)
        env.process(c)
        t = random.expovariate(interval)
        yield env.timeout(t)



In [30]:
def customer(env, name, counter, time_in_bank):
    """Customer arrives, is served and leaves."""
    arrive = env.now
    print('%7.4f %s: Here I am' % (arrive, name))

    with counter.request() as req:
        patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        # Wait for the counter or abort at the end of our tether
        results = yield req | env.timeout(patience)

        wait = env.now - arrive

        if req in results:
            # We got to the counter
            print('%7.4f %s: Waited %6.3f' % (env.now, name, wait))

            tib = random.expovariate(time_in_bank)
            yield env.timeout(tib)
            print('%7.4f %s: Finished' % (env.now, name))

        else:
            # We reneged
            print('%7.4f %s: Fed Up and LEFT the Bank after %6.3f' % (env.now, name, wait))

In [31]:
# Setup and start the simulation
print('Bank Example')
random.seed(RANDOM_SEED)
env = simpy.Environment()


Bank Example


In [28]:
# Start processes and run
service_desk = simpy.Resource(env, capacity=1)
CustList = []
env.process(CustomerGenerator(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, service_desk))
env.run()

 0.0000 Customer00: Here I am
 0.0000 Customer00: Waited  0.000
 0.0268 Customer00: Finished
 0.2040 Customer01: Here I am
 0.2040 Customer01: Waited  0.000
 0.2545 Customer02: Here I am
 0.2981 Customer01: Finished
 0.2981 Customer02: Waited  0.044
 0.3438 Customer02: Finished
 0.7000 Customer03: Here I am
 0.7000 Customer03: Waited  0.000
 0.7060 Customer04: Here I am
 0.7114 Customer05: Here I am
 0.7586 Customer03: Finished
 0.7586 Customer04: Waited  0.053
 0.7794 Customer04: Finished
 0.7794 Customer05: Waited  0.068
 0.8535 Customer05: Finished
 0.9213 Customer06: Here I am
 0.9213 Customer06: Waited  0.000
 1.0579 Customer06: Finished
 1.2529 Customer07: Here I am
 1.2529 Customer07: Waited  0.000
 1.2669 Customer07: Finished
 1.4924 Customer08: Here I am
 1.4924 Customer08: Waited  0.000
 1.5005 Customer08: Finished
 2.1227 Customer09: Here I am
 2.1227 Customer09: Waited  0.000
 2.1431 Customer10: Here I am
 2.1999 Customer09: Finished
 2.1999 Customer10: Waited  0.057
 2.263

## How do we collect meaningful statistics

### For this example we may want to collect the following statistics

* Total number of succesfully processed customers
* Number of customers who fed up and left the branch
* Average waiting time of the customers who left 

An elegant way of collecting such statistics is to define an object and pass it through process chain to collect those statistical values

The following example illustrates that


In [32]:
class Statistics:
    def __init__(self):
        self.no_of_Leaving_cust = 0
        self.list_of_leaving_cust = []
        self.no_of_processed_cust = 0
        self.total_wait_time_of_left_cust = 0

def CustomerGenerator(env, number, interval, counter, sim_stats):
    """This generates customers randomly"""
    for i in range(number):
        c = customer(env, 'Customer%02d' % i, counter, time_in_bank=12.0, sim_stats=sim_stats)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)

def customer(env, name, counter, time_in_bank, sim_stats):
    """Customer arrives, is served and leaves."""
    arrive = env.now

    with counter.request() as req:
        patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        # Wait for the counter or abort at the end of our tether
        results = yield req | env.timeout(patience)
        wait = env.now - arrive

        if req in results:
            # We got to the counter
            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)
            sim_stats.no_of_processed_cust += 1
        else:
            # We reneged
            sim_stats.list_of_leaving_cust.append(name + "waited : " + str(wait) + "and left")
            sim_stats.no_of_Leaving_cust += 1
            sim_stats.total_wait_time_of_left_cust += wait


In [10]:
# Start processes and run
#random.seed(RANDOM_SEED)
NEW_CUSTOMERS = 100  # Total number of customers
service_desk = simpy.Resource(env, capacity=3)
sim_statistics = Statistics()
env.process(CustomerGenerator(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, service_desk, sim_statistics))
env.run()

print ("No of processed customer = ",sim_statistics.no_of_processed_cust)
print ("No of custs leaving = ",sim_statistics.no_of_Leaving_cust)
if (sim_statistics.no_of_Leaving_cust > 0):
    avg = sim_statistics.total_wait_time_of_left_cust / sim_statistics.no_of_Leaving_cust
    print ('Avg wait time for custs leaving = %7.4f' % avg)
else:
    print ('No customers left ..  ')
    

No of processed customer =  91
No of custs leaving =  9
Avg wait time for custs leaving =  2.0141
